<a href="https://colab.research.google.com/github/kyungmin0101/medical_ai/blob/main/240219/Medical_AI_day_4%20Hyunsang%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
import glob
import csv
import pandas as pd
path = "/content/drive/MyDrive/Refractive Index Measurements New/2ndSeries_21.05.21/*/{}*".format('W')
df_list = []
ri_list = []
for file in glob.glob(path, recursive = True) :
  f = open(file,'rt')
  reader = csv.reader(f)
  csv_context = []
  for raw in reader :
    csv_context.append(raw)
  f.close()

  df_list.append(pd.DataFrame(csv_context)[24:])
  ri_list.append(file.split('/')[-2])

In [ ]:
ri_list

In [83]:
ri_list_2=[]
for i in range(len(ri_list)) :
  if 1.3 <= float(ri_list[i]) <= 1.32 :
    ri_list_2.append("CIN1")
  if 1.33 <= float(ri_list[i]) <= 1.37 :
    ri_list_2.append("CIN2")
  if 1.38 <= float(ri_list[i]) <= 1.39 :
    ri_list_2.append("CIN3")
  if float(ri_list[i]) >= 1.4 :
    ri_list_2.append("Cervical Cancer")


In [88]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import numpy as np
import time

DTC = DecisionTreeClassifier()

x_train, x_test, y_train, y_test = train_test_split(df_list, ri_list_2, test_size=0.1)

tree_model = DecisionTreeClassifier(random_state=2000)

nsamples, nx, ny = np.array(x_train).shape
tree_x_train = np.array(x_train).reshape((nsamples,nx*ny))

nsamples, nx, ny = np.array(x_test).shape
tree_x_test = np.array(x_test).reshape((nsamples,nx*ny))

Training_start_time = time.time()
tree_model.fit(tree_x_train, y_train)
Training_end_time = time.time()

Training_execution_time = Training_end_time - Training_start_time



In [97]:
Predict_start_time = time.time()
DTC_pred = tree_model.predict(tree_x_test)
Predict_end_time = time.time()

Predict_execution_time = Predict_end_time - Predict_start_time

accuracy = accuracy_score(y_test, DTC_pred)

print('DTC Acc : {}'.format(accuracy_score(y_test,DTC_pred)))
print(classification_report(y_test, DTC_pred))
print(f"Training Time: {Training_execution_time:.4f} sec")
print(f"Predict Time: {Predict_execution_time:.4f} sec")

DTC Acc : 0.9130434782608695
                 precision    recall  f1-score   support

           CIN1       0.80      1.00      0.89         4
           CIN2       1.00      0.60      0.75         5
           CIN3       0.50      1.00      0.67         1
Cervical Cancer       1.00      1.00      1.00        13

       accuracy                           0.91        23
      macro avg       0.82      0.90      0.83        23
   weighted avg       0.94      0.91      0.91        23

Training Time: 0.7300 sec
Predict Time: 0.1051 sec
